In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv
/kaggle/input/multi-lingual-sentiment-analysis/train.csv
/kaggle/input/multi-lingual-sentiment-analysis/test.csv
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/

In [2]:
import pandas as pd

# Load the train and test data
train_data = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/train.csv')
test_data = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/test.csv')

# Display first few rows of the train and test data
print("Train Data:")
print(train_data.head())

print("\nTest Data:")
print(test_data.head())


Train Data:
   ID                                           sentence     label language
0   1  কর্মীদের ভাল আচরণ এবং খাবারের পাশাপাশি পানীয় ...  Positive       bn
1   2  ગોદરેજ સેન્ટ્રલ એસીમાં તેના કન્ડેન્સર પર 2 વર્...  Positive       gu
2   3  கதைக்களம் பிடித்திருந்தது, அனைத்து நடிகர்களும்...  Positive       ta
3   4  ਵੌਇਸ-ਓਵਰ ਬਹੁਤ ਵਧੀਆ ਸੀ ਅਤੇ ਕਹਾਣੀ ਦੀ ਸੀਮਾ ਵਿੱਚ ਇ...  Positive       pa
4   5  जुथानि थाखाय जायगा गैया। गुबुन मुवा सोग्रा जाय...  Negative       bd

Test Data:
   ID                                           sentence language
0   1                    1120 mAh, ਓਵਰਚਾਰਜਿੰਗ ਦੀ ਸੁਰੱਖਿਆ       pa
1   2  તે સઘન મોઇશ્ચરાઇઝિંગ પ્રદાન કરે છે અને સરસ સ્વ...       gu
2   3                      1120 ಎಂಎಎಚ್, ಮಿತಿಮೀರಿದ ರಕ್ಷಣೆ       kn
3   4  ভাৰতত নিৰ্মিত সৰ্বশ্ৰেষ্ঠ পাৰফিউম ব্ৰেণ্ডবোৰৰ ...       as
4   5  میں نے حال ہی میں "انفولڈ" سے ایک ٹیمپلیٹ خرید...       ur


In [3]:
# Define x_train, y_train, x_test
x_train = train_data['sentence']
y_train = train_data['label']
x_test = test_data['sentence']

# Check the shapes to verify
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")


x_train shape: (1000,)
y_train shape: (1000,)
x_test shape: (100,)


In [4]:
import torch

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
import torch
from transformers import pipeline

# Path to the model directory
model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"

# Create a text-generation pipeline with optimized memory usage
pipeline_generator = pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16},  # Use bfloat16 for reduced memory usage
    device_map="balanced",  # Try balancing between CPU and GPU
    truncation=True,  # Explicitly activate truncation
    pad_token_id=128001  # Ensure padding uses the EOS token (128001)
)


# Test the pipeline with an example input
output = pipeline_generator("Once upon a time,")
print(output)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


[{'generated_text': 'Once upon a time, in a world not so different from our own, there lived a young woman named Sophia. Sophia was'}]


In [6]:
torch.cuda.empty_cache()


In [7]:
def get_sentiment(sentence):
    prompt = f"Please, for god's sake classify the sentiment of the following sentence as Positive or Negative: {sentence}"
    output = pipeline_generator(prompt, max_new_tokens=10, num_return_sequences=1)
    response = output[0]['generated_text'].strip().lower()
    # Parse the output to return "Positive" or "Negative"
    if "positive" in response:
        return "Positive"
    else:
        return "Negative"

In [8]:
torch.cuda.empty_cache()


In [9]:
    # predictions = []
    
    # # Loop through each row in the test_data and get the sentiment of the sentence
    # for _, row in test_data.iterrows():
    #     sentence = row['sentence']  # Assuming the column name containing sentences is 'sentence'
    #     sentiment = get_sentiment(sentence)
    #     predictions.append(sentiment)
    
    # # Create a new DataFrame for the submission
    # submission_df = pd.DataFrame({
    #     'ID': test_data['ID'],  # Assuming 'ID' column exists in the test data
    #     'label': predictions
    # })
    
    # # Save the submission to a CSV file
    # submission_df.to_csv('submission.csv', index=False)
    
    # print("Submission file created successfully.")

In [10]:
sentences = test_data['sentence'].tolist()

# Process the sentences in batches while using the custom prompt
def batch_get_sentiment_with_prompt(sentences, batch_size=32):
    results = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        
        # Prepare the prompts for each sentence
        prompts = [f"Please, for god's sake classify the sentiment of the following sentence as Positive or Negative: {sentence}" for sentence in batch]
        
        # Get the model's output for all the prompts in one batch
        batch_output = pipeline_generator(prompts, max_new_tokens=10, num_return_sequences=1)

        # Process the output and classify sentiment
        for output in batch_output:
            sentiment = output[0]['generated_text'].strip().lower()
            if "positive" in sentiment:
                results.append("Positive")
            else:
                results.append("Negative")
    
    return results

# Get the sentiments in batches using the custom prompt engineering
sentiments = batch_get_sentiment_with_prompt(sentences)

# Create the submission DataFrame
submission_df = pd.DataFrame({
    'ID': test_data['ID'],
    'label': sentiments
})

# Save the results to a CSV file
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!
